In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib import gridspec 
import random
import xgboost as xgb
from sklearn import preprocessing
from math import sqrt
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
top = np.percentile(train.Purchase,99.5)
train = train[ (train['Purchase']<top) ]
train.fillna(-999,inplace=True)
test.fillna(-999,inplace=True)
train.dtypes

User_ID                         int64
Product_ID                     object
Gender                         object
Age                            object
Occupation                      int64
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                  int64
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchase                        int64
dtype: object

In [2]:
train['Marital_Status'] = train['Marital_Status'].astype(str)
test['Marital_Status'] = test['Marital_Status'].astype(str)

train['Occupation'] = train['Occupation'].astype(str)
test['Occupation'] = test['Occupation'].astype(str)

In [3]:
user_id_to_category_map = {}
customer_purchase_power = train.groupby("User_ID")["Purchase"].sum()
#customer_purchase_power.median()
values = customer_purchase_power.iteritems()
perc1 = np.percentile(customer_purchase_power,100)

In [4]:
for key, val in values:
    if val <= 143662.0:
        user_id_to_category_map[key] = 143662
    elif val <= 202156.0:
        user_id_to_category_map[key] = 202156
    elif val <= 275408.0:
        user_id_to_category_map[key] = 275408
    elif val <= 378451.0:
        user_id_to_category_map[key] = 378451
    elif val <= 513874.0:
        user_id_to_category_map[key] = 513874
    elif val <= 687241.0:
        user_id_to_category_map[key] = 687241
    elif val <= 934097.0:
        user_id_to_category_map[key] = 934097
    elif val <= 1332584.0:
        user_id_to_category_map[key] = 133258
    elif val <= 2029891.0:
        user_id_to_category_map[key] = 2029891
    else:
        user_id_to_category_map[key] = 10400703
customer_purchase_power.median()
def get_customer_category(user_id):
    if user_id in user_id_to_category_map:
        return user_id_to_category_map[user_id]
    return 5
train["Purchasing_power"] = list(map(lambda user_id: get_customer_category(user_id), train["User_ID"]))
test["Purchasing_power"] = list(map(lambda user_id: get_customer_category(user_id), test["User_ID"]))

In [5]:
train["UID_Min"] = train.groupby(['User_ID'])['Purchase'].transform('min')
userID_min_dict = train.groupby(['User_ID'])['Purchase'].min().to_dict()
test['UID_Min'] = test['User_ID'].apply(lambda x:userID_min_dict.get(x,0))

train["UID_MaxP"] = train.groupby(['User_ID'])['Purchase'].transform('max')
userID_max_dict = train.groupby(['User_ID'])['Purchase'].max().to_dict()
test['UID_MaxP'] = test['User_ID'].apply(lambda x:userID_max_dict.get(x,0))

train["PID_MinP"] = train.groupby(['Product_ID'])['Purchase'].transform('min')
productID_min_dict = train.groupby(['Product_ID'])['Purchase'].min().to_dict()
test['PID_MinP'] = test['Product_ID'].apply(lambda x:productID_min_dict.get(x,0))

train["PID_MaxP"] = train.groupby(['Product_ID'])['Purchase'].transform('max')
productID_max_dict = train.groupby(['Product_ID'])['Purchase'].max().to_dict()
test['PID_MaxP'] = test['Product_ID'].apply(lambda x:productID_max_dict.get(x,0))

train["User_ID_Count"] = train.groupby(['User_ID'])['User_ID'].transform('count')
uID_count = train.groupby(['User_ID']).size().to_dict()
test['User_ID_Count'] = test['User_ID'].apply(lambda x:uID_count.get(x,0))

train["Product_ID_Count"] = train.groupby(['Product_ID'])['Product_ID'].transform('count')
pID_count = train.groupby(['Product_ID']).size().to_dict()
test['Product_ID_Count'] = test['Product_ID'].apply(lambda x:pID_count.get(x,0))

train["UID_MeanP"] = train.groupby(['User_ID'])['Purchase'].transform('mean')
userID_mean_dict = train.groupby(['User_ID'])['Purchase'].mean().to_dict()
test['UID_MeanP'] = test['User_ID'].apply(lambda x:userID_mean_dict.get(x,0))

train["PID_MeanP"] = train.groupby(['Product_ID'])['Purchase'].transform('mean')
productID_mean_dict = train.groupby(['Product_ID'])['Purchase'].mean().to_dict()
test['PID_MeanP'] = test['Product_ID'].apply(lambda x:productID_mean_dict.get(x,0))

train["Pc1_MeanP"] = train.groupby(['Product_Category_1'])['Purchase'].transform('mean')
pc1_mean_dict = train.groupby(['Product_Category_1'])['Purchase'].mean().to_dict()
test['Pc1_MeanP'] = test['Product_Category_1'].apply(lambda x:pc1_mean_dict.get(x,0))

train["Pc1_MinP"] = train.groupby(['Product_Category_1'])['Purchase'].transform('min')
pc1_min_dict = train.groupby(['Product_Category_1'])['Purchase'].min().to_dict()
test['Pc1_MinP'] = test['Product_Category_1'].apply(lambda x:pc1_min_dict.get(x,0))

train["Pc1_MaxP"] = train.groupby(['Product_Category_1'])['Purchase'].transform('max')
pc1_max_dict = train.groupby(['Product_Category_1'])['Purchase'].max().to_dict()
test['Pc1_MaxP'] = test['Product_Category_1'].apply(lambda x:pc1_max_dict.get(x,0))

train["Pc2_MeanP"] = train.groupby(['Product_Category_2'])['Purchase'].transform('mean')
pc2_mean_dict = train.groupby(['Product_Category_2'])['Purchase'].mean().to_dict()
test['Pc2_MeanP'] = test['Product_Category_2'].apply(lambda x:pc2_mean_dict.get(x,0))

train["Pc2_MinP"] = train.groupby(['Product_Category_2'])['Purchase'].transform('min')
pc2_min_dict = train.groupby(['Product_Category_2'])['Purchase'].min().to_dict()
test['Pc2_MinP'] = test['Product_Category_2'].apply(lambda x:pc2_min_dict.get(x,0))

train["Pc2_MaxP"] = train.groupby(['Product_Category_2'])['Purchase'].transform('max')
pc2_max_dict = train.groupby(['Product_Category_2'])['Purchase'].max().to_dict()
test['Pc2_MaxP'] = test['Product_Category_2'].apply(lambda x:pc2_max_dict.get(x,0))

train["Pc3_MeanP"] = train.groupby(['Product_Category_3'])['Purchase'].transform('mean')
pc3_mean_dict = train.groupby(['Product_Category_3'])['Purchase'].mean().to_dict()
test['Pc3_MeanP'] = test['Product_Category_3'].apply(lambda x:pc3_mean_dict.get(x,0))

train["Pc3_MinP"] = train.groupby(['Product_Category_3'])['Purchase'].transform('min')
pc3_min_dict = train.groupby(['Product_Category_3'])['Purchase'].min().to_dict()
test['Pc3_MinP'] = test['Product_Category_3'].apply(lambda x:pc3_min_dict.get(x,0))

train["Pc3_MaxP"] = train.groupby(['Product_Category_3'])['Purchase'].transform('max')
pc3_max_dict = train.groupby(['Product_Category_3'])['Purchase'].max().to_dict()
test['Pc3_MaxP'] = test['Product_Category_3'].apply(lambda x:pc3_max_dict.get(x,0))

userID_25p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['UID_25Perc'] = train['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))
test['UID_25Perc'] = test['User_ID'].apply(lambda x:userID_25p_dict.get(x,0))

userID_50p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,50.)).to_dict()
train['UID_50Perc'] = train['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))
test['UID_50Perc'] = test['User_ID'].apply(lambda x:userID_50p_dict.get(x,0))

userID_75p_dict = train.groupby(['User_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['UID_75Perc'] = train['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))
test['UID_75Perc'] = test['User_ID'].apply(lambda x:userID_75p_dict.get(x,0))

productID_25p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['PID_25Perc'] = train['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))
test['PID_25Perc'] = test['Product_ID'].apply(lambda x:productID_25p_dict.get(x,0))

productID_50p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['PID_50Perc'] = train['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))
test['PID_50Perc'] = test['Product_ID'].apply(lambda x:productID_50p_dict.get(x,0))

productID_75p_dict = train.groupby(['Product_ID'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['PID_75Perc'] = train['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))
test['PID_75Perc'] = test['Product_ID'].apply(lambda x:productID_75p_dict.get(x,0))

pc1_25p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Pc1_25Perc'] = train['Product_Category_1'].apply(lambda x:pc1_25p_dict.get(x,0))
test['Pc1_25Perc'] = test['Product_Category_1'].apply(lambda x:pc1_25p_dict.get(x,0))

pc1_50p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Pc1_50Perc'] = train['Product_Category_1'].apply(lambda x:pc1_50p_dict.get(x,0))
test['Pc1_50Perc'] = test['Product_Category_1'].apply(lambda x:pc1_50p_dict.get(x,0))

pc1_75p_dict = train.groupby(['Product_Category_1'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Pc1_75Perc'] = train['Product_Category_1'].apply(lambda x:pc1_75p_dict.get(x,0))
test['Pc1_75Perc'] = test['Product_Category_1'].apply(lambda x:pc1_75p_dict.get(x,0))


pc2_25p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Pc2_25Perc'] = train['Product_Category_2'].apply(lambda x:pc2_25p_dict.get(x,0))
test['Pc2_25Perc'] = test['Product_Category_2'].apply(lambda x:pc2_25p_dict.get(x,0))

pc2_50p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Pc2_50Perc'] = train['Product_Category_2'].apply(lambda x:pc2_50p_dict.get(x,0))
test['Pc2_50Perc'] = test['Product_Category_2'].apply(lambda x:pc2_50p_dict.get(x,0))

pc2_75p_dict = train.groupby(['Product_Category_2'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Pc2_75Perc'] = train['Product_Category_2'].apply(lambda x:pc2_75p_dict.get(x,0))
test['Pc2_75Perc'] = test['Product_Category_2'].apply(lambda x:pc2_75p_dict.get(x,0))

pc3_25p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,25)).to_dict()
train['Pc3_25Perc'] = train['Product_Category_3'].apply(lambda x:pc3_25p_dict.get(x,0))
test['Pc3_25Perc'] = test['Product_Category_3'].apply(lambda x:pc3_25p_dict.get(x,0))

pc3_50p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,50)).to_dict()
train['Pc3_50Perc'] = train['Product_Category_3'].apply(lambda x:pc3_50p_dict.get(x,0))
test['Pc3_50Perc'] = test['Product_Category_3'].apply(lambda x:pc3_50p_dict.get(x,0))

pc3_75p_dict = train.groupby(['Product_Category_3'])['Purchase'].apply(lambda x:np.percentile(x,75)).to_dict()
train['Pc3_75Perc'] = train['Product_Category_3'].apply(lambda x:pc3_75p_dict.get(x,0))
test['Pc3_75Perc'] = test['Product_Category_3'].apply(lambda x:pc3_75p_dict.get(x,0))

train["Occupation_Count"] = train.groupby(['Occupation'])['Occupation'].transform('count')
occupation_count_dict = train.groupby(['Occupation']).size().to_dict()
test['Occupation_Count'] = test['Occupation'].apply(lambda x:occupation_count_dict.get(x,0))

train["Age_Count"] = train.groupby(['Age'])['Age'].transform('count')
age_count_dict = train.groupby(['Age']).size().to_dict()
test['Age_Count'] = test['Age'].apply(lambda x:age_count_dict.get(x,0))

train["Product_Category_1_Count"] = train.groupby(['Product_Category_1'])['Product_Category_1'].transform('count')
pc1_count_dict = train.groupby(['Product_Category_1']).size().to_dict()
test['Product_Category_1_Count'] = test['Product_Category_1'].apply(lambda x:pc1_count_dict.get(x,0))

train["Product_Category_2_Count"] = train.groupby(['Product_Category_2'])['Product_Category_2'].transform('count')
pc2_count_dict = train.groupby(['Product_Category_2']).size().to_dict()
test['Product_Category_2_Count'] = test['Product_Category_2'].apply(lambda x:pc2_count_dict.get(x,0))

train["Product_Category_3_Count"] = train.groupby(['Product_Category_3'])['Product_Category_3'].transform('count')
pc3_count_dict = train.groupby(['Product_Category_3']).size().to_dict()
test['Product_Category_3_Count'] = test['Product_Category_3'].apply(lambda x:pc3_count_dict.get(x,0))

In [6]:
cat1 = pd.DataFrame()
cat1['User_ID'] = test['User_ID']
cat1['Product_ID'] = test['Product_ID']
cat_col=["User_ID", "Product_ID"]
for var in cat_col:
    lb = preprocessing.LabelEncoder()
    full_var_data = pd.concat((train[var], test[var]), axis=0).astype('str')
    lb.fit(full_var_data)
    train[var] = lb.transform(train[var].astype('str'))
    test[var] = lb.transform(test[var].astype('str'))
train_y = train["Purchase"]
train.drop(["Purchase"], axis=1, inplace=True)

In [7]:
train['User_ID'] = train['User_ID'].astype(str)
test['User_ID'] = test['User_ID'].astype(str)

train['Product_ID'] = train['Product_ID'].astype(str)
test['Product_ID'] = test['Product_ID'].astype(str)

In [8]:
train.dtypes


User_ID                        object
Product_ID                     object
Gender                         object
Age                            object
Occupation                     object
City_Category                  object
Stay_In_Current_City_Years     object
Marital_Status                 object
Product_Category_1              int64
Product_Category_2            float64
Product_Category_3            float64
Purchasing_power                int64
UID_Min                         int64
UID_MaxP                        int64
PID_MinP                        int64
PID_MaxP                        int64
User_ID_Count                   int64
Product_ID_Count                int64
UID_MeanP                     float64
PID_MeanP                     float64
Pc1_MeanP                     float64
Pc1_MinP                        int64
Pc1_MaxP                        int64
Pc2_MeanP                     float64
Pc2_MinP                        int64
Pc2_MaxP                        int64
Pc3_MeanP   

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, train_y, test_size=0.04, random_state=42)

In [10]:
import catboost as cb
'''params = {'depth': [8, 9, 10],
          'learning_rate' : [0.03, 0.1, 0.15],
         'l2_leaf_reg': [1,4,9],
         'iterations': [500]} '''
cfi = np.where(train.dtypes != np.float)[0]
#cfi=[train['Age'], train['Gender'], train['City_Category'], train['Stay_In_Current_City_Years']]
from catboost import CatBoostRegressor
model=CatBoostRegressor(iterations=4000, depth=10,learning_rate=0.04, loss_function='RMSE')
model.fit(X_train, y_train,cat_features=cfi,eval_set=(X_test, y_test),plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 4796.2632168	test: 4805.5705636	best: 4805.5705636 (0)	total: 1.31s	remaining: 1h 27m 4s
1:	learn: 4663.3980584	test: 4673.4424448	best: 4673.4424448 (1)	total: 2.67s	remaining: 1h 29m 8s
2:	learn: 4536.4960017	test: 4547.0094951	best: 4547.0094951 (2)	total: 4.03s	remaining: 1h 29m 26s
3:	learn: 4416.0869058	test: 4427.2333576	best: 4427.2333576 (3)	total: 5.32s	remaining: 1h 28m 31s
4:	learn: 4301.6801631	test: 4313.8380148	best: 4313.8380148 (4)	total: 6.71s	remaining: 1h 29m 24s
5:	learn: 4192.8786199	test: 4205.2264109	best: 4205.2264109 (5)	total: 8.05s	remaining: 1h 29m 18s
6:	learn: 4090.6682486	test: 4103.6908670	best: 4103.6908670 (6)	total: 9.36s	remaining: 1h 29m 1s
7:	learn: 3993.6991632	test: 4007.3823328	best: 4007.3823328 (7)	total: 10.7s	remaining: 1h 28m 54s
8:	learn: 3901.7316671	test: 3915.8984612	best: 3915.8984612 (8)	total: 11.9s	remaining: 1h 28m 14s
9:	learn: 3814.9996313	test: 3829.6249855	best: 3829.6249855 (9)	total: 13.3s	remaining: 1h 28m 21s
10:

81:	learn: 2490.2867906	test: 2520.3544458	best: 2520.3544458 (81)	total: 1m 42s	remaining: 1h 21m 41s
82:	learn: 2488.5773955	test: 2518.5392086	best: 2518.5392086 (82)	total: 1m 44s	remaining: 1h 21m 52s
83:	learn: 2486.8915875	test: 2516.7117433	best: 2516.7117433 (83)	total: 1m 45s	remaining: 1h 22m 2s
84:	learn: 2485.4581990	test: 2515.2876935	best: 2515.2876935 (84)	total: 1m 47s	remaining: 1h 22m 15s
85:	learn: 2483.9168391	test: 2513.5572781	best: 2513.5572781 (85)	total: 1m 49s	remaining: 1h 22m 48s
86:	learn: 2482.5266571	test: 2512.2044946	best: 2512.2044946 (86)	total: 1m 50s	remaining: 1h 23m 3s
87:	learn: 2481.0274739	test: 2510.5882186	best: 2510.5882186 (87)	total: 1m 52s	remaining: 1h 23m 38s
88:	learn: 2479.7677969	test: 2509.2706130	best: 2509.2706130 (88)	total: 1m 54s	remaining: 1h 23m 57s
89:	learn: 2478.5313557	test: 2507.9835578	best: 2507.9835578 (89)	total: 1m 56s	remaining: 1h 24m 17s
90:	learn: 2477.4143573	test: 2506.7481731	best: 2506.7481731 (90)	total: 1

160:	learn: 2439.2951943	test: 2468.7958494	best: 2468.7958494 (160)	total: 4m 19s	remaining: 1h 43m 10s
161:	learn: 2438.9858507	test: 2468.6221285	best: 2468.6221285 (161)	total: 4m 22s	remaining: 1h 43m 32s
162:	learn: 2438.6545022	test: 2468.3676564	best: 2468.3676564 (162)	total: 4m 24s	remaining: 1h 43m 42s
163:	learn: 2438.3171408	test: 2468.0692510	best: 2468.0692510 (163)	total: 4m 27s	remaining: 1h 44m 8s
164:	learn: 2438.0609113	test: 2467.9560984	best: 2467.9560984 (164)	total: 4m 29s	remaining: 1h 44m 12s
165:	learn: 2437.6021783	test: 2467.5464237	best: 2467.5464237 (165)	total: 4m 30s	remaining: 1h 44m 12s
166:	learn: 2437.2688092	test: 2467.2626935	best: 2467.2626935 (166)	total: 4m 32s	remaining: 1h 44m 15s
167:	learn: 2436.9460317	test: 2466.8830550	best: 2466.8830550 (167)	total: 4m 35s	remaining: 1h 44m 33s
168:	learn: 2436.6415414	test: 2466.7815508	best: 2466.7815508 (168)	total: 4m 37s	remaining: 1h 44m 59s
169:	learn: 2436.3998117	test: 2466.5353094	best: 2466.5

239:	learn: 2418.7106137	test: 2454.9727079	best: 2454.9727079 (239)	total: 7m 10s	remaining: 1h 52m 21s
240:	learn: 2418.4987676	test: 2454.8366835	best: 2454.8366835 (240)	total: 7m 12s	remaining: 1h 52m 21s
241:	learn: 2418.0920875	test: 2454.6625565	best: 2454.6625565 (241)	total: 7m 14s	remaining: 1h 52m 24s
242:	learn: 2417.9216289	test: 2454.5607795	best: 2454.5607795 (242)	total: 7m 16s	remaining: 1h 52m 26s
243:	learn: 2417.6759856	test: 2454.5109866	best: 2454.5109866 (243)	total: 7m 18s	remaining: 1h 52m 31s
244:	learn: 2417.5676150	test: 2454.4486344	best: 2454.4486344 (244)	total: 7m 20s	remaining: 1h 52m 26s
245:	learn: 2417.3754995	test: 2454.3840361	best: 2454.3840361 (245)	total: 7m 22s	remaining: 1h 52m 27s
246:	learn: 2417.1807497	test: 2454.2615459	best: 2454.2615459 (246)	total: 7m 25s	remaining: 1h 52m 47s
247:	learn: 2416.9817184	test: 2454.1629578	best: 2454.1629578 (247)	total: 7m 27s	remaining: 1h 52m 51s
248:	learn: 2416.8243635	test: 2454.0792168	best: 2454.

...

3620:	learn: 2074.3119936	test: 2385.9604216	best: 2385.8580665 (3609)	total: 2h 28m 50s	remaining: 15m 34s
3621:	learn: 2074.2366383	test: 2385.9575656	best: 2385.8580665 (3609)	total: 2h 28m 53s	remaining: 15m 32s
3622:	learn: 2074.1162014	test: 2385.9501070	best: 2385.8580665 (3609)	total: 2h 28m 56s	remaining: 15m 29s
3623:	learn: 2074.0284556	test: 2385.9454028	best: 2385.8580665 (3609)	total: 2h 28m 59s	remaining: 15m 27s
3624:	learn: 2073.9661516	test: 2385.9565505	best: 2385.8580665 (3609)	total: 2h 29m 1s	remaining: 15m 25s
3625:	learn: 2073.8704372	test: 2385.9565253	best: 2385.8580665 (3609)	total: 2h 29m 3s	remaining: 15m 22s
3626:	learn: 2073.7855746	test: 2385.9270003	best: 2385.8580665 (3609)	total: 2h 29m 7s	remaining: 15m 20s
3627:	learn: 2073.6153123	test: 2385.9388415	best: 2385.8580665 (3609)	total: 2h 29m 10s	remaining: 15m 17s
3628:	learn: 2073.5507299	test: 2385.9422007	best: 2385.8580665 (3609)	total: 2h 29m 12s	remaining: 15m 15s
3629:	learn: 2073.5309864	test:

3697:	learn: 2068.5837737	test: 2385.9372847	best: 2385.8107811 (3681)	total: 2h 32m 37s	remaining: 12m 27s
3698:	learn: 2068.5603746	test: 2385.9244727	best: 2385.8107811 (3681)	total: 2h 32m 40s	remaining: 12m 25s
3699:	learn: 2068.5211936	test: 2385.8999946	best: 2385.8107811 (3681)	total: 2h 32m 42s	remaining: 12m 22s
3700:	learn: 2068.4012407	test: 2385.8970281	best: 2385.8107811 (3681)	total: 2h 32m 46s	remaining: 12m 20s
3701:	learn: 2068.3646765	test: 2385.8944623	best: 2385.8107811 (3681)	total: 2h 32m 48s	remaining: 12m 18s
3702:	learn: 2068.2584138	test: 2385.8838957	best: 2385.8107811 (3681)	total: 2h 32m 52s	remaining: 12m 15s
3703:	learn: 2068.1639533	test: 2385.8876071	best: 2385.8107811 (3681)	total: 2h 32m 54s	remaining: 12m 13s
3704:	learn: 2068.0523924	test: 2385.8604520	best: 2385.8107811 (3681)	total: 2h 32m 59s	remaining: 12m 10s
3705:	learn: 2067.9468388	test: 2385.8137868	best: 2385.8107811 (3681)	total: 2h 33m 4s	remaining: 12m 8s
3706:	learn: 2067.9049407	test

3774:	learn: 2062.7404776	test: 2385.6803784	best: 2385.6803784 (3774)	total: 2h 36m 34s	remaining: 9m 19s
3775:	learn: 2062.6777382	test: 2385.6912336	best: 2385.6803784 (3774)	total: 2h 36m 36s	remaining: 9m 17s
3776:	learn: 2062.5874431	test: 2385.6859831	best: 2385.6803784 (3774)	total: 2h 36m 40s	remaining: 9m 15s
3777:	learn: 2062.5073442	test: 2385.7008227	best: 2385.6803784 (3774)	total: 2h 36m 44s	remaining: 9m 12s
3778:	learn: 2062.4071961	test: 2385.6882758	best: 2385.6803784 (3774)	total: 2h 36m 48s	remaining: 9m 10s
3779:	learn: 2062.2975075	test: 2385.7626414	best: 2385.6803784 (3774)	total: 2h 36m 51s	remaining: 9m 7s
3780:	learn: 2062.2374466	test: 2385.7606027	best: 2385.6803784 (3774)	total: 2h 36m 54s	remaining: 9m 5s
3781:	learn: 2062.1654563	test: 2385.7368795	best: 2385.6803784 (3774)	total: 2h 36m 57s	remaining: 9m 2s
3782:	learn: 2061.9676567	test: 2385.7536933	best: 2385.6803784 (3774)	total: 2h 37m	remaining: 9m
3783:	learn: 2061.9364817	test: 2385.7502584	bes

3851:	learn: 2056.8638343	test: 2385.6277913	best: 2385.5243607 (3844)	total: 2h 40m 17s	remaining: 6m 9s
3852:	learn: 2056.8096820	test: 2385.6150232	best: 2385.5243607 (3844)	total: 2h 40m 21s	remaining: 6m 7s
3853:	learn: 2056.7672215	test: 2385.6058286	best: 2385.5243607 (3844)	total: 2h 40m 24s	remaining: 6m 4s
3854:	learn: 2056.7280948	test: 2385.5956465	best: 2385.5243607 (3844)	total: 2h 40m 27s	remaining: 6m 2s
3855:	learn: 2056.6921451	test: 2385.5920390	best: 2385.5243607 (3844)	total: 2h 40m 30s	remaining: 5m 59s
3856:	learn: 2056.6477741	test: 2385.6020215	best: 2385.5243607 (3844)	total: 2h 40m 33s	remaining: 5m 57s
3857:	learn: 2056.6294254	test: 2385.5908274	best: 2385.5243607 (3844)	total: 2h 40m 35s	remaining: 5m 54s
3858:	learn: 2056.5346152	test: 2385.5660017	best: 2385.5243607 (3844)	total: 2h 40m 37s	remaining: 5m 52s
3859:	learn: 2056.4827025	test: 2385.5742206	best: 2385.5243607 (3844)	total: 2h 40m 39s	remaining: 5m 49s
3860:	learn: 2056.4300207	test: 2385.5545

3928:	learn: 2051.5803335	test: 2385.3266096	best: 2385.2650749 (3914)	total: 2h 43m 54s	remaining: 2m 57s
3929:	learn: 2051.5295591	test: 2385.3129521	best: 2385.2650749 (3914)	total: 2h 43m 57s	remaining: 2m 55s
3930:	learn: 2051.4469722	test: 2385.3147957	best: 2385.2650749 (3914)	total: 2h 44m	remaining: 2m 52s
3931:	learn: 2051.3138002	test: 2385.3063920	best: 2385.2650749 (3914)	total: 2h 44m 3s	remaining: 2m 50s
3932:	learn: 2051.2497031	test: 2385.3149937	best: 2385.2650749 (3914)	total: 2h 44m 6s	remaining: 2m 47s
3933:	learn: 2051.2321903	test: 2385.3201531	best: 2385.2650749 (3914)	total: 2h 44m 9s	remaining: 2m 45s
3934:	learn: 2051.1509469	test: 2385.2993986	best: 2385.2650749 (3914)	total: 2h 44m 12s	remaining: 2m 42s
3935:	learn: 2051.0960697	test: 2385.2844909	best: 2385.2650749 (3914)	total: 2h 44m 14s	remaining: 2m 40s
3936:	learn: 2050.9657196	test: 2385.3099948	best: 2385.2650749 (3914)	total: 2h 44m 17s	remaining: 2m 37s
3937:	learn: 2050.8732740	test: 2385.2417427

In [11]:
pred=model.predict(test)


In [12]:
cat1['Purchase'] = pred
#sum(n < 0 for n in cat1["Purchase"].values.flatten())
cat1[cat1["Purchase"] < 0] = 0
cat1.to_csv('catb1.csv',index=False)

In [13]:
model.get_feature_importance(data=None,
                       
                       prettified=True,
                       thread_count=-1,
                       verbose=True)

,Feature Id,Importances
0,PID_MeanP,12.875408
1,PID_75Perc,6.350930
2,UID_25Perc,5.294287
3,PID_50Perc,5.038760
4,UID_MeanP,4.845355
5,UID_75Perc,4.667984
6,PID_25Perc,4.663049
7,User_ID,4.597544
8,UID_50Perc,3.986670
9,UID_MaxP,3.732204


In [15]:
model.get_feature_importance(prettified=False,
                       thread_count=-1,
                       verbose=True)

array([ 4.78661476,  2.56110512,  0.82752408,  2.31638368,  2.35369155,
        2.00005454,  1.7011587 ,  0.33287349,  2.52420593,  0.34065044,
        0.33802206,  0.68653398,  0.73845411,  1.19087589,  1.3545928 ,
        0.7442668 ,  3.43596306,  2.06592483,  2.87041141,  3.96507071,
        6.67602475,  2.57202487,  1.6451261 , 16.95221985,  2.70276005,
        6.35805618,  4.90295405,  4.98040604,  5.83584354,  0.62616754,
        0.89778799,  0.66333251,  0.24053712,  0.18651432,  0.13547205,
        0.2108478 ,  0.22014846,  0.20305793,  0.73223388,  1.23121201,
        0.86472605,  0.09171381,  1.22776814,  1.03567359,  0.10116278,
        0.74948195,  0.82236868])